# Einführung
Regression als Methode erscheint zunächst verwandt mit der Klassifikation. Während bei letzterer allerdings nur eine diskrete Menge von Ergebnissen in Betracht zu ziehen ist (nämlich genau die zur Auswahl stehenden Klassen), zielt die Regression auf stetige Ergebnisse. Gefunden werden soll nämlich ein funktionaler Zusammenhang zwischen verschiedenen Attributen, sodass die Ausgabe eine reelle Zahl darstellt.

## Inhaltsverzeichnis
- [Vorbetrachtungen](#Vorbetrachtungen)
- [Beispiel 1](#Beispiel-1)
- [Beispiel 2](#Beispiel-2)
- [Beispiel 3](#Beispiel-3)

## Vorbetrachtungen
Zunächst kommen wir aber noch einmal zu den Voraussetzungen: Zur Regression wird eine Menge von Punkten $G = \{(x_1, y_1), \dots, (x_n, y_n)\}$ benötigt, deren Dimensionen sich in zwei Kategorien einteilen lassen.

1. Ein Wert $x$ ist eine **unabhängige** Variable. (Im weiteren Verlauf erweitern wir diese Betrachtung auch auf höhere Dimensionen.)
2. Ein Wert $y$ ist eine **abhängige** Variable, da sie mit Hilfe eines funktionalen Zusammenhangs aus $x$ berechnet werden kann.

Bevor wir jedoch mit einem Beispiel beginnen, benötigen wir einige Bibliotheken:

In [ ]:
import random

from ipywidgets import interact, IntSlider, FloatSlider
import pandas as pd
import numpy as np
import plotly.express as px

from tui_dsg.regression import draw_regression, R_squared
from tui_dsg.datasets import heightweight_path

Hinweis: In vielen Zellen wird entweder per `random.seed` oder `np.random.seed` ein Startwert für den Zufallsgenerator vergeben. Das garantiert, dass die zufälligen Werte nicht "entarten". Zum Testen können Sie diese Zeile natürlich auch auskommentieren oder den Startwert verändern.

## Beispiel 1
Die folgende Zelle erzeugt einen künstlichen Beispieldatensatz, der Körpergröße und Gewicht verschiedener Personen enthält und dessen funktionaler Zusammenhang offensichtlich ist.

In [ ]:
random.seed(1)

df1 = pd.DataFrame({
    'Größe': (random.random() * 60 + 140 for _ in range(80))
})
df1['Gewicht'] = 0.75 * df1['Größe'] - 60

px.scatter(df1, x='Größe', y='Gewicht')

Sie erkennen auch ohne eine Analyse des dahinterstehenden Codes, dass sich alle Punkte exakt entlang einer Geraden bewegen. Den funktionalen Zusammenhang zu finden ist deshalb in diesem Fall trivial: Man wählt zwei unterschiedliche Punkte, bestimmt die Steigung mithilfe eines Steigungsdreiecks und zuletzt noch die Verschiebung entlang der $y$-Achse:

In [ ]:
# zwei Punkte wählen
x1, y1 = df1.iloc[0]
x2, y2 = df1.iloc[79]

# Anstieg über Steigungsdreieck berechnen
m = (y2 - y1) / (x2 - x1)

# Verschiebung berechnen
n = y1 - m * x1

m, n

Grafisch dargestellt durchläuft die berechnete Gerade genau dem Datensatz:

In [ ]:
fig = px.scatter(df1, x='Größe', y='Gewicht')
draw_regression(fig, n, m)

fig

## Beispiel 2
Reale Werte unterliegen allerdings immer leichten Abweichungen.

Zum einen können statistische Annahmen über das der Messung zugrunde liegende Merkmal getroffen werden: Nicht jeder Mensch, der $180cm$ groß ist, wiegt exakt $75kg$. Stattdessen befinden sich die meisten Menschen in der Nähe dieses Wertes, während sehr wenige Personen mit dieser Körpergröße nur $50kg$ wiegen.

Zum anderen sind Messfehler nicht auszuschließen, während außerdem mit Rauschen in der Messung gerechnet werden muss. Menschen könnten beim Ablesen der Größe nicht perfekt gerade stehen oder der Blickwinkel auf den Maßstab ändert sich immer wieder leicht.

Die folgende Zelle simuliert daher eine normalverteilte Abweichung.

In [ ]:
np.random.seed(1)

df2 = df1.copy()
df2['Gewicht'] += np.random.normal(loc=0, scale=3, size=80)

px.scatter(df2, x='Größe', y='Gewicht')

Die Wahl der Punkte zur Bestimmung des Steigungsdreiecks hat nun einen immensen Einfluss auf das Ergebnis.

In [ ]:
fig = px.scatter(df2, x='Größe', y='Gewicht')

for p1, p2 in ((0, 24), (25, 49), (50, 74)):
    x1, y1 = df2.iloc[p1]
    x2, y2 = df2.iloc[p2]
    m = (y2 - y1) / (x2 - x1)
    n = y1 - m * x1

    draw_regression(fig, n, m)

fig

Um zu bewerten, welche die besten Punkte zur Berechnung sind, müsste im Voraus also schon die Ausgleichsgerade bekannt sein. In einigen Fällen liegt aber sogar kein einziger Punkt des Datensatzes auf der *optimalen* Geraden, was die Methode des Steigungsdreiecks unmöglich macht.

## Beispiel 3

Das letzte Beispiel entstammt einem [gebrauchsfertigen Datensatz](https://www.kaggle.com/datasets/burnoutminer/heights-and-weights-dataset) und enthält die Körpergröße und das Gewicht von $25.000$ Personen, von denen wir zur Darstellung $250$ auswählen. Die vertikalen Linien, die in der nachfolgenden Darstellung jeweils die Differenz zwischen dem erwarteten und dem tatsächlichen $y$-Wert markieren, werden als Residuen bezeichnet und im nächsten Abschnitt detailliert behandelt. Die quadratische Summe dieser zu minimieren, wird Optimierungskriterium für die nachfolgenden Verfahren.

In [ ]:
df3 = pd.read_csv(heightweight_path, index_col='index').sample(250, random_state=5)

fig = px.scatter(df3, x='height', y='weight')
draw_regression(fig, -30, 0.5, residual_opacity=1)

fig

In der letzten Zelle in diesem Notebook haben Sie noch einmal die Möglichkeit, beide Parameter experimentell anzupassen. Beobachten Sie währenddessen die quadrierte Summe der Residuen und versuchen Sie, ein möglichst geringes Ergebnis zu erzielen.

In [ ]:
@interact(
    m=IntSlider(min=-1000, max=1000, step=1, value=-30),
    n=FloatSlider(min=0.1, max=3, step=0.05, value=0.5)
)
def _(m, n):
    R = R_squared(df3['height'], df3['weight'], m, n)
    fig = px.scatter(df3, x='height', y='weight', title=f'ΣR² = {R:.2f}')
    draw_regression(fig, m, n)

    return fig